In [5]:
from IPython.display import display
import pandas as pd 
import pickle
import os 
from utils.env import EvoD4jEnv
import re 

project = "Lang"
version = ""
ts_id = "newTS"
prompt_no = 1
example_num = 2
try_no = ""
mut = ""

def class_name_to_test_path(class_name):
    return class_name.replace('.', '/')+'.java'


# Get Ground Truth
evo_test_df_all = [] 
# To make it start with index 1
evo_test_df_all.append(pd.DataFrame())
for i in range(1, 66):
    if i == 2 or i == 30 or i == 63:
        evo_test_df_all.append(pd.DataFrame())
        continue # deprecated version
    env = EvoD4jEnv(project, i, ts_id)
    # Original
    evo_test_path = env.evosuite_test_dir.replace("/root/workspace","/Users/hslee/Workspace/AutoOracle/workspace")
    with open(os.path.join(evo_test_path, "evo_tests_df.pkl"),'rb') as f:
        evo_test_df = pickle.load(f)
        evo_test_df["ground_truth"] = "correct"
    # Filtering the failing tests
    with open(os.path.join(evo_test_path, "failing_tests_on_fixed")) as f:
        data = f.readlines()
    for l in data:
        if l.startswith('---'):
            pattern = r"--- (.*?)::(.*?)$"
            match = re.search(pattern, l)
            if match:
                class_name = match.group(1)
                class_name = class_name.replace('.', '/')+'.java'
                test_no = match.group(2)
            evo_test_df.loc[(evo_test_df["evo_relpath"] == class_name) & (evo_test_df["evo_test_no"] == test_no),"ground_truth"] = "incorrect"
    evo_test_df_all.append(evo_test_df)


    

## Parsing Chat_Reply 


In [6]:
# Parsing Chat_reply with prompt 1 
for i in range(1, 66):
    if i == 2 or i==30 or i == 63 :
        continue # deprecated version or compilation error
    env = EvoD4jEnv(project, i, ts_id)
    chat_reply_dir_root = env.evosuite_chat_reply_dir.replace("/root/workspace","/Users/hslee/Workspace/AutoOracle/workspace")

    #with prompt1 & example2
    for j in range(1,4):
        chat_reply_dir = os.path.join(chat_reply_dir_root,"prompt1/example2/try{}".format(j))
        chat_reply_list = os.listdir(chat_reply_dir)

        for chat_reply_f in chat_reply_list:
            with open(os.path.join(chat_reply_dir, chat_reply_f),'r') as fr:
                data = fr.read()
            split_chat_reply_f = chat_reply_f.strip().split('_')
            dir = split_chat_reply_f[0].replace('.','/')
            test_no = split_chat_reply_f[1]
            df = evo_test_df_all[i]
            if re.search("incorrect", data):
                df.loc[ (df["dir"] == dir) & (df["evo_test_no"] == test_no), "predict{}".format(j) ] = "incorrect"
            elif re.search("correct", data):
                df.loc[ (df["dir"] == dir) & (df["evo_test_no"] == test_no), "predict{}".format(j)  ] = "correct" 
    


## RESULT

In [7]:

from sklearn import metrics

for j in range(1,4):
    ground_truth =[]
    predict=[]
    print("***********************")
    print("Result of prediction {}".format(j))
    print("***********************")
    for i in range(1, 66):
        if i == 2 or i==30 or i == 63 :
            continue # deprecated version or compilation error
        tmp_ground_truth = evo_test_df_all[i]["ground_truth"].values.tolist()
        tmp_ground_truth = [ 1 if x == "correct" else 0 for x in tmp_ground_truth]
        tmp_predict = evo_test_df_all[i]["predict{}".format(j)].values.tolist()
        tmp_predict = [ 1 if x == "correct" else 0 for x in tmp_predict]
        ground_truth.extend(tmp_ground_truth)
        predict.extend(tmp_predict)

    tn, fp, fn, tp = metrics.confusion_matrix(ground_truth, predict).ravel()

    print("True Negative:", tn)
    print("False Positive:", fp)
    print("False Negative:", fn)
    print("True Positive:", tp)
    print()
    accuracy = metrics.accuracy_score(ground_truth, predict)
    print("accuracy:", accuracy)

    precision = metrics.precision_score(ground_truth, predict)
    print("precision:", precision)

    recall = metrics.recall_score(ground_truth, predict)
    print("recall:", recall)


    f1 = metrics.f1_score(ground_truth, predict)
    print("f1:", f1)
    print(" ")

***********************
Result of prediction 1
***********************
True Negative: 24
False Positive: 20
False Negative: 836
True Positive: 1016

accuracy: 0.5485232067510548
precision: 0.9806949806949807
recall: 0.5485961123110151
f1: 0.7036011080332409
 
***********************
Result of prediction 2
***********************
True Negative: 28
False Positive: 16
False Negative: 866
True Positive: 986

accuracy: 0.5348101265822784
precision: 0.9840319361277445
recall: 0.5323974082073434
f1: 0.6909600560616678
 
***********************
Result of prediction 3
***********************
True Negative: 26
False Positive: 18
False Negative: 855
True Positive: 997

accuracy: 0.5395569620253164
precision: 0.9822660098522168
recall: 0.5383369330453563
f1: 0.6955005231949773
 


In [9]:
print("***********************")
print("Consistency_Original")
print("***********************")
ground_truth =[]
predict=[]
for i in range(1, 66):
    if i == 2 or i==30 or i == 63 :
        continue # deprecated version or compilation error
    evo_test_df_all[i] = evo_test_df_all[i].loc[evo_test_df_all[i]["predict1"] == evo_test_df_all[i]["predict2"],:]
    evo_test_df_all[i] = evo_test_df_all[i].loc[evo_test_df_all[i]["predict2"] == evo_test_df_all[i]["predict3"],:]
    tmp_ground_truth = evo_test_df_all[i]["ground_truth"].values.tolist()
    tmp_ground_truth = [ 1 if x == "correct" else 0 for x in tmp_ground_truth]
    tmp_predict1 = evo_test_df_all[i]["predict1"].values.tolist()
    tmp_predict2 = evo_test_df_all[i]["predict2"].values.tolist()
    tmp_predict3 = evo_test_df_all[i]["predict3"].values.tolist()
    tmp_predict = [ 1 if x == ("correct","correct","correct") else 0 for x in zip(tmp_predict1,tmp_predict2,tmp_predict3)]
    ground_truth.extend(tmp_ground_truth)
    predict.extend(tmp_predict)

tn, fp, fn, tp = metrics.confusion_matrix(ground_truth, predict).ravel()

print("True Negative:", tn)
print("False Positive:", fp)
print("False Negative:", fn)
print("True Positive:", tp)

accuracy = metrics.accuracy_score(ground_truth, predict)
print("accuracy:", accuracy)

precision = metrics.precision_score(ground_truth, predict)
print("precision:", precision)

recall = metrics.recall_score(ground_truth, predict)
print("recall:", recall)

f1 = metrics.f1_score(ground_truth, predict)
print("f1:", f1)

print(tn+ fp+ fn+ tp)

***********************
Consistency_Original
***********************
True Negative: 11
False Positive: 3
False Negative: 191
True Positive: 591
accuracy: 0.7562814070351759
precision: 0.9949494949494949
recall: 0.7557544757033248
f1: 0.8590116279069767
796
